In [1]:
%load_ext autoreload
%autoreload 2
%aimport -torch
%aimport -torchvision
%aimport -PIL

import torch
import torchvision
import PIL

import utils
import datasets
from models import SRCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
div2k = datasets.Div2K(32, 2)
dataloader = torch.utils.data.DataLoader(div2k, batch_size=16)

model = SRCNN()

In [3]:
b = None
for i, batch in enumerate(dataloader):
    print(i)
    if i > 3:
        break
    b = batch

0
1
2
3
4


In [9]:
div2k.test()

tensor(0.2706)


0.14812612533569336